# Basic Usage of the `tyssue` library

## Easy creation of a 2D epithelial sheet

In [ ]:
%matplotlib inline

# Core object
from tyssue import Sheet
# Simple 2D geometry
from tyssue import PlanarGeometry as geom
# Visualisation
from tyssue.draw import sheet_view

sheet = Sheet.planar_sheet_2d('basic2D', nx=6, ny=7,
                              distx=1, disty=1)
geom.update_all(sheet)



In [ ]:
fig, ax = sheet_view(sheet, mode="2D")
fig.set_size_inches(8, 8)

We can have a cleaner, better order `sheet` with the `sanitize` method:


In [ ]:
# Give the tissue a nice hear cut ;)
sheet.sanitize(trim_borders=True, order_edges=True)
geom.update_all(sheet)

In [ ]:
fig, ax = sheet_view(sheet, mode="2D")
fig.set_size_inches(8, 8)

### A remark on the half-edge data structure


As is represented in the above graph, each edge between two cells is composed of two half-edges (only one half-edge is present in the border ones). This makes it easier to compute many cell-specific quantities, as well as keeping a well oriented mesh. This is inspired by CGAL [polyhedral surfaces](https://doc.cgal.org/4.2/CGAL.CGAL.3D-Polyhedral-Surface/html/index.html).

## Datasets and specifications


The data associated with the mesh displayed above, i.e. the points positions,
the connectivity information, etc. is stored in pandas [DataFrame](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe) objects, hold together in the `datasets` dictionnary.


Depending on the geometry, the following dataframes are populated:
* `datasets["edge"]` or `sheet.edge_df`: The edge related dataframe contains 
 - the connectivity information: source and target vertices, associated face and (for thick tissues)
   the associated cell body.
 - geometry data associated with the edge, such as its length
 - any suplemental data, such as a color or a dynamical parameter (an elasticity for example)

* `datasets["vert"]` or `sheet.vert_df`: The vertices related dataframe. In the apical junction mesh above, 
  those are  the vertices at the cells junctions. It usually holds the coordinates of the points, 
  and geometry or dynamical data.

* `datasets["face"]` or `sheet.face_df`: The faces related dataframe. For a thin, 2D tissue, this corresponds to
  a cell of the epithelium, delimited by its edges. In thick, 3D models, one cell has several faces
  (the apical, sagittal and basal ones for a 3D monolayer, for example).
 
* `datasets["cell"]` or `sheet.cell_df`: The cells related dataframe, only for 3D, thick, epithelium.
  Each cell have several faces.
  
  
  ![datastructures in `tyssue`](../illus/tyssue_data_management.png)
 

In [ ]:
for element, data in sheet.datasets.items():
    print(element, ':', data.shape)

In [ ]:
sheet.datasets['edge'].head()

The `edge_df` dataframe contains most of the information. In particular, each time the geometry is updated with the `geom.update_all(sheet)` function, the positions of the source and target vertices of each edge are copied to the `"sx", "sy"` and `"tx", "ty"` columns, respectively.



In [ ]:
sheet.face_df.head()

We can use all the goodies from pandas DataFrames objects. For example, it is possible to 
compute the average edge length for each face like so:

In [ ]:
sheet.edge_df.groupby('face')['length'].mean().head()

**Specifications** are defined as a nested dictionnary, `sheet.specs`. For each element, the specification defines the columns of the corresonding DataFrame and their default values. An extra key at the root of the specification is called `"settings"`, and can hold specific parameters, for example the arguments for an energy minimization procedure.

In [ ]:
sheet.specs

It is possible to update dynamically the data structure. For example, let's assume we want to put quasi-static model to minimize energy for the tyssue. For this we need three things.
* A specification dictionnary with the required data
* A model
* A solver

In [ ]:
from tyssue.config.dynamics import quasistatic_plane_spec
from tyssue.dynamics.planar_vertex_model import PlanarModel
from tyssue.solvers import QSSolver

# Update the specs
sheet.update_specs(quasistatic_plane_spec())

# Find energy minimum
solver = QSSolver()
res = solver.find_energy_min(sheet, 
                             geom,
                             PlanarModel)

print("Successfull gradient descent? ", res['success'])
fig, ax = sheet_view(sheet)

## Upcasting and downcasting data


### Upcasting

Geometry or physics computations often require to access for example
the cell related data on each of the cell's edges. The `Epithelium`
class and its derivatives defines utilities to make this,
i.e copying the area of each face to each of its edges:

In [ ]:
print('Faces associated with the first edges:')
print(sheet.edge_df['face'].head())
print('\n')

# First edge associated face
face = sheet.edge_df.loc[0, 'face']

print('Area of cell # {}:'.format(int(face)))
print(sheet.face_df.loc[face, 'area'])

print('\n')
print('Upcasted areas over the edges:')
print(sheet.upcast_face(sheet.face_df['area']).head())

The values have indeed be upcasted. This can also be done with the source and target vertices 
(`sheet.upcast_srce`, `sheet.upcast_trgt`) and cells in the 3D case (`sheet.upcast_cell`).

### Downcasting

This is usually done by `groupby` operations as shown above.
Syntactic sugar is available for summation, e.g. over every edges with a given source: 

In [ ]:
sheet.sum_srce(sheet.edge_df['line_tension']).head()

## Input and Output

The 'native' format is to save the datasets to hdf5 via [`pandas.HDFStore`](https://pandas.pydata.org/pandas-docs/stable/cookbook.html#hdfstore). The `io.obj` also provides functions to export the junction mesh or triangulations to the wavefront `OBJ` format (requires `vispy`), for easy import in 3D software such as Blender.

Here is the code to save the data in wavefront `OBJ`:
```python
obj.save_junction_mesh('junctions.obj', sheet)
```

The standard data format for the datasets is HDF:

In [ ]:
from tyssue.io import hdf5

#### Writing

In [ ]:
hdf5.save_datasets('tmp_data.hdf5', sheet)


#### Reading

In [ ]:
dsets = hdf5.load_datasets('tmp_data.hdf5')
sheet2 = Sheet('reloaded', dsets)

In [ ]:
!rm tmp_data.hdf5

Specs can be saved as json files:

In [ ]:
import json

with open("tmp_specs.json", "w") as jh:
    json.dump(sheet.specs, jh)

And reloaded accordingly

In [ ]:
with open("tmp_specs.json", "r") as jh:
    specs = json.load(jh)

sheet2.update_specs(specs, reset=False)

In [ ]:
!rm tmp_specs.json